In [21]:
import matplotlib.pyplot as plt
import numpy as np

def v_cal(time):
    v0 = 10    #初始速度
    v1 = 23    #中间速度
    v2 = 15    #最终速度
    t_v0_v1 = 3;    #初始速度到中间速度持续时间
    t_v1_v2 = 3;    #中间速度到最终速度持续时间
    if(time < t_v0_v1):
        return v0 + time*(v1 - v0)/t_v0_v1
    elif(time < t_v1_v2):
        return v0 + time*(v2 - v1)/t_v1_v2
    else:
        return v1 + time*(v2 - v1)/t_v1_v2
    
    
SSA_NUM = 6
SS_NUM = SSA_NUM*3
WH_NUM = 18
WHR_NUM = WH_NUM * 2
T_1US = 0.000001
T_10US =  0.00001
sx = []    #传感器x坐标
wx = []    #车轴x坐标
ds = []    #传感器间距
dw = []    #轴距
lwn = [0 for i in range(SS_NUM)]    #每个传感器的丢轴个数
rwn = [0 for i in range(SS_NUM)]    #每个传感器的多轴个数
swn = [WH_NUM for i in range(SS_NUM)]    #每个传感器经过的实际轮轴数，加上多轴，减去丢轴
ws = [0 for i in range(SS_NUM)]    #每个轴经过的传感器数

dt =  T_10US
t = [[0]*SS_NUM for i in range(WHR_NUM)]    #绝对时刻表
dwst = [[0]*SS_NUM for i in range(WHR_NUM)]    #相对时刻表
lost_ax = [[0]*SS_NUM for i in range(WH_NUM)]    #丢轴标志表
red_ax = [[0]*SS_NUM for i in range(WH_NUM)]    #多轴时间表
ws_order = [[0]*SS_NUM for i in range(WH_NUM)]

#丢轴设置
lost_ax[10][9] = 1;
lost_ax[12][9] = 1;
lost_ax[10][11] = 1;
    
#多轴设置
red_ax[7][6] = T_10US*5000;
red_ax[14][6] = T_10US*5000;
#red_ax[16][6] = T_10US*5000;
red_ax[2][9] = T_10US;

#sx = 0.0
#wx[0] = -1.0
ds = [0.295,0.360,4.150,0.327,0.350,4.150,0.354,0.235,4.150,0.210,0.350,4.150,0.206,0.344,4.150,0.223,0.345]
dw = [1.873,7.979,1.864,2.989,1.863,7.946,1.869,3.043,1.858,7.955,1.843,3.785,2.031,1.992,4.689,2.043,1.977]
#print(ds[0])
sx.append(0)
wx.append(0)
for i in range(0,SS_NUM-1):
    sx.append(sx[i]+ds[i])
for i in range(0,WH_NUM-1):
    wx.append(wx[i]-dw[i])
for i in range(0,SS_NUM-1):
    swn.append(WH_NUM)
for i in range(0,WH_NUM):
    ws.append(0)
time = 0
v = 23
#t.append(0)
ws_order.append(0)
order = 0
while(wx[WH_NUM-1] < sx[SS_NUM-1] + 30 * dt):
    time += dt;
    v = v_cal(time)
    for i in range(0,WH_NUM):
        wx[i] += dt * v
    for i in range(0,WH_NUM):
        for j in range(ws[i],SS_NUM):
            if wx[i] > sx[j]:
                ws[i] += 1
                t[i][j] = time
                ws_order[i][j] = order
                order += 1
                
#for i in range(0,WH_NUM):
#    print('')
#    for j in range(0,SS_NUM):
#        print('%8f ' % t[i][j],end = '')
        
#for i in range(0,WH_NUM):
#    print('')
#    for j in range(0,SS_NUM):
#        print('%4d ' % ws_order[i][j],end = '')

        
#//处理丢轴多轴
for j in range(0,SS_NUM):
    for i in range(0,WH_NUM):
        if lost_ax[i][j]:
            t[i][j] = 0
            swn[j] -= 1
            lwn[j] += 1
    for i in range(WH_NUM-1,0,-1):
        if red_ax[i][j] > T_1US:
            swn[j] += 1
            rwn[j] += 1
            for k in range(WH_NUM-1+rwn[j],i,-1):
                t[k][j] = t[k-1][j]
            t[i+1][j] = t[i][j] + red_ax[i][j]
print('')
print(t)
#%matplotlib qt
#fig1 = plt.figure('fig1')
#plt.grid(True)
#plt.xlim((0,SS_NUM))
#plt.ylim((0,max(t[max(swn)])))
#plt.xticks(np.arange(0,SS_NUM,1))
#plt.yticks(np.arange(0,4,0.1))
#plt.plot(list(map(list,(zip(*t)))),'.')
#plt.plot(list(map(list,(zip(*t))))[0:max(swn)],'.')
#fig1.show()

#//整理丢轴
for j in range(0,SS_NUM):
    for i in range(0,WH_NUM-1+rwn[j]):
        if(t[i][j] < T_1US):
            for k in range(i,WH_NUM-1+rwn[j]):
                t[k][j] = t[k+1][j]

#//计算周期除了第0行后面的数据
for j in range(0,SS_NUM):
    for i in range(0,swn[j]):
        dwst[i+1][j] = t[i+1][j] - t[i][j]
#//计算第0行数据
for j in range(0,SS_NUM-1):
    dwst[0][j+1] = t[0][j+1] -  t[0][j]
dwst[0][0] = 0

for j in range(0,SS_NUM):
    print('S%d,N%d t:' % (j,swn[j]),end = '')
    for i in range(0,swn[j]):
        print('%7f ' % t[i][j],end = '')
    print('')
        
        
%matplotlib qt
fig2 = plt.figure('absolute time table')
plt.grid(True)
plt.xlim((0,SS_NUM))
plt.ylim(0,max(t[WH_NUM-1]))
plt.xticks(np.arange(0,SS_NUM,1))
plt.yticks(np.arange(0,max(t[WH_NUM-1]),0.1))
plt.plot(list(map(list,(zip(*t)))),'.')
fig2.show()
#print(dwst)
#for j in range(0,SS_NUM):
#    for i in range(0,swn[j]):
#        print('t[%d][%d]=%10f ' % (i,j,t[i][j]))
        
#print(rwn)
#for j in range(SS_NUM-1,-1,-1):
#    print(j)
#for i in range(0,WH_NUM):
    #print(i,wx[i])

#%matplotlib qt
#plt.plot(t[0:max(swn)],'.')
#plt.grid(True)
#plt.show()




[[1e-05, 0.02942999999999897, 0.06516000000000743, 0.46308000000032234, 0.4934200000003527, 0.5257500000002421, 0.8977199999985492, 0.9285499999984089, 0.9489399999983161, 1.299830000000048, 1.3171500000001615, 1.34592000000035, 1.6790500000025324, 1.6952100000026382, 1.7221400000028146, 2.039980000004897, 2.0567100000050065, 2.0825300000051756], [0.18454000000004378, 0.21311000000007235, 0.24782000000010707, 0.6350399999997447, 0.6646199999996101, 0.6961299999994667, 1.0593299999984724, 1.08946999999867, 1.1093999999988005, 1.4528900000010507, 1.469860000001162, 1.4980600000013466, 1.8248400000034875, 1.8407100000035914, 1.8671500000037646, 2.1794600000058106, 2.1959100000059184, 2.2213000000060847], [0.916899999998462, 0.9425299999983453, 0.9736799999982035, 1.3234000000002024, 1.3502700000003784, 1.3789200000005661, 1.7106400000027393, 1.7382900000029204, 1.7566000000030404, 2.073110000005114, 2.0888000000052167, 2.1148900000053876, 2.4181900000073746, 2.4329600000074714, 2.4575800

In [4]:
print(max(swn))
print(max(t[WH_NUM-1]))

20
3.606440000015159
